In [ ]:
import PyPDF2
from openai import OpenAI
import faiss
import numpy as np
import openai
import os

In [ ]:
# extract text from research papers (pdf format)
# TODO: only extract meaningful portions
def extract_text(path):
    text = ""

    with open(path, 'rb') as f:
        reader = PyPDF2.PdfReader(f)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text


In [ ]:
paper_dirs = os.listdir('papers')

In [ ]:
len(paper_dirs)

11

In [ ]:
papers = []
for fp in paper_dirs:
    papers.append(extract_text(f'papers/{fp}'))


unknown widths : 
[0, IndirectObject(812, 0, 13518862000)]
unknown widths : 
[0, IndirectObject(814, 0, 13518862000)]
unknown widths : 
[0, IndirectObject(816, 0, 13518862000)]


In [ ]:
max_tokens = 512
overlap_tokens = 50

In [ ]:
# chunk text by tokens to prepare for embedding
# TODO: figure out why NLTK isn't working
def chunk_text(text):
    words = text.split()
    chunks = []
    i = 0

    while i < len(words):
        chunk = words[i:i + max_tokens]
        chunk_text = ' '.join(chunk)
        chunks.append(chunk_text)

        i += max_tokens - overlap_tokens

    return chunks

In [ ]:
chunks = []
for paper in papers:
    chunks.append(chunk_text(paper))

In [ ]:
client = OpenAI()

In [ ]:
def get_embedding(text):
   text = ' '.join(text.split('\n'))
   return client.embeddings.create(input = [text], model="text-embedding-3-small").data[0].embedding

In [ ]:
# make embeddings for all papers and group them together
# TODO: research if is this the right approach or if we should separate them
embeddings = []
for chunk in chunks:
    for block in chunk:
        embeddings.append(get_embedding(block))

In [ ]:
len(embeddings)

325

In [ ]:
f_embeddings = np.array(embeddings).astype('float32')

In [ ]:
# TODO: do more research on faiss metrics, using cosine product for now
index = faiss.IndexFlatIP(len(embeddings[0]))
index.add(f_embeddings)

In [ ]:
query = 'What metrics are there for quantifying intimacy as it relates to LLMS? Please give me 2 detailed, actionable metrics that I can use on textual data as well as how to measure them.'

In [ ]:
query_embedding = get_embedding(query)

In [ ]:
d, idx = index.search(np.array([query_embedding]), 5)

In [ ]:
f_chunks = [item for sublist in chunks for item in sublist]

In [ ]:
top_chunks = [f_chunks[i] for i in idx[0]]

In [ ]:
context = '\n'.join(top_chunks)
prompt = f"Question: {query}\nContext: {context}\nAnswer:"

In [ ]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an expert in intimacy, especially as it relates to LLMs."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=800,
    temperature=0.7
)

In [ ]:
answer = response.choices[0].message.content

In [ ]:
print(answer)

Two detailed, actionable metrics for quantifying intimacy in textual data related to LLMs are topic analysis and linguistic hedging analysis.

1. **Topic Analysis**: Topic analysis involves examining the content and themes of the text to gauge the level of intimacy conveyed. Intimate topics tend to involve personal or emotionally charged subjects that foster a sense of closeness and connection between individuals. To measure the intimacy level using topic analysis:
   - **How to Measure**: Use topic modeling techniques such as Latent Dirichlet Allocation (LDA) to extract topics from the textual data. Assign each question a probability distribution over different topics based on the words and phrases used. Then, analyze the topics that are more likely to be associated with intimacy, such as personal experiences, emotions, relationships, etc.
   - **Actionable Steps**: 
     - Implement LDA or a similar topic modeling algorithm on the textual data.
     - Identify topics that are frequen

Two detailed, actionable metrics for quantifying intimacy in textual data related to LLMs are topic analysis and linguistic hedging analysis.

1. **Topic Analysis**: Topic analysis involves examining the content and themes of the text to gauge the level of intimacy conveyed. Intimate topics tend to involve personal or emotionally charged subjects that foster a sense of closeness and connection between individuals. To measure the intimacy level using topic analysis:
   - **How to Measure**: Use topic modeling techniques such as Latent Dirichlet Allocation (LDA) to extract topics from the textual data. Assign each question a probability distribution over different topics based on the words and phrases used. Then, analyze the topics that are more likely to be associated with intimacy, such as personal experiences, emotions, relationships, etc.
   - **Actionable Steps**:
     - Implement LDA or a similar topic modeling algorithm on the textual data.
     - Identify topics that are frequently present in questions rated as intimate.
     - Calculate the distribution of intimacy levels across different topics to understand which topics are more intimate.

2. **Linguistic Hedging Analysis**: Linguistic hedging refers to the use of language that softens the impact of a statement or question, making it more tentative or polite. In the context of intimacy, certain linguistic cues, such as hedging or uncertainty markers, can indicate a lower level of intimacy. To measure the level of intimacy using linguistic hedging analysis:
   - **How to Measure**: Identify linguistic features that typically signal hedging, such as modal verbs (e.g., "could," "might"), hedges (e.g., "kind of," "sort of"), or indirect language constructions. Quantify the frequency and intensity of these hedging markers in the text.
   - **Actionable Steps**:
     - Create a list of hedging markers and uncertainty indicators relevant to the context of intimacy.
     - Utilize natural language processing tools to detect and count the occurrences of these hedging markers in the textual data.
     - Compare the presence of hedging markers in questions rated as less intimate versus more intimate to understand their impact on perceived intimacy levels.

By employing these metrics and associated measurement techniques, you can gain insights into the level of intimacy conveyed in textual data related to LLMs and further analyze how individuals modulate their language to align with social norms and interpersonal dynamics.

In [ ]:
print(answer)

Two actionable metrics for quantifying intimacy in textual data related to LLMs are linguistic hedging frequency and personal disclosure depth.

1. **Linguistic Hedging Frequency:**
   - **Description:** Linguistic hedging refers to the use of language that softens the impact of statements or questions, indicating uncertainty or cautiousness. In intimate conversations, individuals may use fewer hedges as they feel more comfortable and open with their communication partner.
   - **Measurement:** To measure linguistic hedging frequency, you can analyze the text for hedging words and phrases such as "maybe," "perhaps," "kind of," "sort of," etc. You can create a list of common hedging terms and use natural language processing tools like spaCy or NLTK to identify and count their occurrences in the text data.
   - **Action:** Calculate the ratio of hedging terms to total words in each text sample to quantify the level of linguistic hedging. A lower ratio may indicate higher intimacy in the 

Two actionable metrics for quantifying intimacy in textual data related to LLMs are linguistic hedging frequency and personal disclosure depth.

1. **Linguistic Hedging Frequency:**
   - **Description:** Linguistic hedging refers to the use of language that softens the impact of statements or questions, indicating uncertainty or cautiousness. In intimate conversations, individuals may use fewer hedges as they feel more comfortable and open with their communication partner.
   - **Measurement:** To measure linguistic hedging frequency, you can analyze the text for hedging words and phrases such as "maybe," "perhaps," "kind of," "sort of," etc. You can create a list of common hedging terms and use natural language processing tools like spaCy or NLTK to identify and count their occurrences in the text data.
   - **Action:** Calculate the ratio of hedging terms to total words in each text sample to quantify the level of linguistic hedging. A lower ratio may indicate higher intimacy in the communication.

2. **Personal Disclosure Depth:**
   - **Description:** Personal disclosure depth refers to the extent to which individuals share personal information, emotions, or experiences in their communication. In intimate conversations, individuals tend to engage in deeper levels of personal disclosure.
   - **Measurement:** To measure personal disclosure depth, you can analyze the text for the richness and depth of personal information shared, including details about feelings, experiences, or personal history. You can use sentiment analysis tools to identify emotional content and topic modeling techniques to explore the depth of personal topics discussed.
   - **Action:** Assign a score or rating to each text sample based on the depth of personal disclosure present. This can be a subjective judgment or a quantitative measure based on the amount of personal content shared.

By tracking these metrics in textual data, you can gain insights into the level of intimacy conveyed in LLM communication and identify patterns related to linguistic cues and personal sharing behaviors.

In [ ]:
revised_query = '''What metrics are available to quantify intimacy in text generated by LLMs?
                Please provide 2 detailed, actionable metrics that can be applied to textual data.
                For each metric, explain how to measure it step-by-step, and if possible, include example measurements or
                considerations on how it applies specifically to LLM outputs.'''

In [ ]:
revised_query_embedding = get_embedding(revised_query)

In [ ]:
d, idx = index.search(np.array([revised_query_embedding]), 5)

In [ ]:
revised_top_chunks = [f_chunks[i] for i in idx[0]]

In [ ]:
context = '\n'.join(revised_top_chunks)
prompt = f"Question: {revised_query}\nContext: {context}\nAnswer:"

In [ ]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are an expert in the field of intimacy and human-computer interaction, "
                "especially focusing on how intimacy is measured in text generated by large language models (LLMs). "
                "Provide responses that are clear, structured, and focus on practical metrics with actionable steps "
                "for implementation."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=800,
    temperature=0.7
)

In [ ]:
revised_answer = response.choices[0].message.content

In [ ]:
print(revised_answer)

### Actionable Metric 1: Intimacy Score based on Self-Disclosure

**Step-by-Step Measurement:**
1. **Identify Self-Disclosure Cues:** Look for personal details shared by the author about themselves or their life, including feelings, goals, dreams, fears, etc.
2. **Quantify Self-Disclosure:** Assign a score to each self-disclosure cue based on the level of intimacy it conveys (e.g., factual vs. emotional self-disclosure).
3. **Aggregate Scores:** Sum up the scores of all self-disclosure cues in the text to calculate the overall intimacy score.

**Example Measurement:**  
In a text generated by an LLM, identify instances of self-disclosure such as personal experiences, emotions, or vulnerabilities. Assign a score of 1-5 to each self-disclosure cue based on its depth of intimacy. Calculate the total score to quantify the level of intimacy expressed in the text.

**Considerations for LLM Outputs:**  
LLMs can generate text with varying degrees of self-disclosure. By quantifying self-disclo

### Actionable Metric 1: Intimacy Score based on Self-Disclosure

**Step-by-Step Measurement:**
1. **Identify Self-Disclosure Cues:** Look for personal details shared by the author about themselves or their life, including feelings, goals, dreams, fears, etc.
2. **Quantify Self-Disclosure:** Assign a score to each self-disclosure cue based on the level of intimacy it conveys (e.g., factual vs. emotional self-disclosure).
3. **Aggregate Scores:** Sum up the scores of all self-disclosure cues in the text to calculate the overall intimacy score.

**Example Measurement:**  
In a text generated by an LLM, identify instances of self-disclosure such as personal experiences, emotions, or vulnerabilities. Assign a score of 1-5 to each self-disclosure cue based on its depth of intimacy. Calculate the total score to quantify the level of intimacy expressed in the text.

**Considerations for LLM Outputs:**  
LLMs can generate text with varying degrees of self-disclosure. By quantifying self-disclosure cues and assigning intimacy scores, this metric can provide a structured way to measure intimacy in LLM-generated text.

---

### Actionable Metric 2: Intimacy Level based on Linguistic Markers

**Step-by-Step Measurement:**
1. **Identify Linguistic Markers:** Look for specific terms or phrases that create a sense of closeness and connection between the author and the reader.
2. **Analyze Linguistic Patterns:** Examine the frequency and context of these markers to gauge the overall intimacy level of the text.
3. **Assign Intimacy Levels:** Based on the presence and intensity of linguistic markers, categorize the text into intimacy levels (e.g., low, medium, high).

**Example Measurement:**  
Scan an LLM-generated text for linguistic markers like inclusive pronouns (e.g., "we," "us"), emotional adjectives, or empathetic phrases. Analyze the distribution and usage of these markers to determine the intimacy level of the text as low, medium, or high.

**Considerations for LLM Outputs:**  
LLMs can exhibit patterns in language use that reflect varying levels of intimacy. By focusing on linguistic markers associated with closeness and connection, this metric offers a practical approach to quantifying intimacy in LLM-generated text.

---

These metrics provide actionable steps to measure intimacy in text generated by LLMs, offering a structured framework for assessing the level of intimacy conveyed in the language model outputs.
